In [29]:
import numpy as np
from lookup_table import CaseNum2EdgeOffset, getCaseNum
import trimesh
import os
import time
import matplotlib.pyplot as plt
import mcubes
# import pyvista as pv

In [30]:
def visialize_pc(pc):
    # 生成带高度映射的颜色
    colors = plt.cm.jet(pc[:,2])  # 使用jet色映射根据Z值着色

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # 绘制带颜色映射的点云
    sc = ax.scatter(pc[:,0], pc[:,1], pc[:,2], 
                    c=colors, s=20, alpha=0.6, 
                    depthshade=True, marker='o')

    # 设置视角参数
    ax.view_init(elev=90, azim=-90)  # 仰角25度，方位角-45度
    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')
    ax.set_zlabel('Z Axis')
    plt.title('3D Point Cloud with Height Coloring')

    # 添加颜色条
    cbar = plt.colorbar(sc, shrink=0.5)
    cbar.set_label('Height Value')

    plt.tight_layout()
    plt.show()


In [31]:
def marching_cube(thres,cells):
    print(cells.shape)
    # vertices use dictionary to avoid duplicate axes
    vertex_array = {}
    face_array = []
    t1 = time.time()
    # -------------------TODO------------------ 
    # compute vertices and faces
    # vertices: [N, 3]
    # faces: [M, 3], e.g. np.array([[0,1,2]]) means a triangle composed of vertices[0], vertices[1] and vertices[2]
    # for-loop is allowed to reduce difficulty
    # -------------------TODO------------------     
    # 遍历所有体素单元（排除边界防止越界）
    for x in range(cells.shape[0]-1):
        for y in range(cells.shape[1]-1):
            for z in range(cells.shape[2]-1):
                case_nums = getCaseNum(x,y,z,thres,cells)
                
                edges = []
                for edge_num in case_nums:
                    if edge_num == -1:
                        break
                    edges.append(edge_num)
                
                for i in range(0, len(edges), 3):
                    if i+2 >= len(edges):
                        break
                    
                    triangle_vertices = []
                    for j in range(3):
                        edge_idx = edges[i+j]
                        edge_info = CaseNum2EdgeOffset[edge_idx]
                        
                        a_x, a_y, a_z = edge_info[0], edge_info[1], edge_info[2]
                        b_x, b_y, b_z = edge_info[3], edge_info[4], edge_info[5]
                        
                        val_a = cells[x+a_x, y+a_y, z+a_z]
                        val_b = cells[x+b_x, y+b_y, z+b_z]
                        
                        t = (thres - val_a) / (val_b - val_a) if val_a != val_b else 0.5
                        x_coord = x + a_x + t*(b_x - a_x)
                        y_coord = y + a_y + t*(b_y - a_y)
                        z_coord = z + a_z + t*(b_z - a_z)
                        
                        coord_key = (round(x_coord,4), round(y_coord,4), round(z_coord,4))
                        
                        if coord_key not in vertex_array:
                            vertex_array[coord_key] = len(vertex_array)
                        triangle_vertices.append(vertex_array[coord_key])
                    
                    face_array.append(triangle_vertices)
    t2 = time.time()
    # for vertex in vertex_array:
    #     print(vertex)
    # print(vertex_array.keys())
    print("\nTime taken by algorithm\n"+'-'*40+"\n{} s".format(t2-t1))
    vertex_array = list(vertex_array.keys())
    print(len(vertex_array), len(face_array))
    return np.array(vertex_array), np.array(face_array)

In [32]:
# reconstruct these two animals
shape_name_lst = ['spot', 'bob']
for shape_name in shape_name_lst:
    data = np.load(os.path.join('data', shape_name + '_cell.npy'))
    verts, faces = marching_cube(0, data)
    print(verts.shape, faces.shape)
    # visialize_pc(verts)
    mesh = trimesh.Trimesh(vertices=verts, faces=faces, process=False)
    mesh.show()
    
    mesh_txt = trimesh.exchange.obj.export_obj(mesh)
    with open(os.path.join('../results', shape_name + '.obj'),"w") as fp:
        fp.write(mesh_txt)



(64, 64, 64)

Time taken by algorithm
----------------------------------------
4.093505144119263 s
6858 13712
(6858, 3) (13712, 3)
(64, 64, 64)

Time taken by algorithm
----------------------------------------
3.755760669708252 s
8968 17936
(8968, 3) (17936, 3)
